# Fine Tuning DistilBERT for MultiLabel Text Classification - multilingual

In [9]:
import warnings
warnings.simplefilter('ignore')
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn import metrics
import transformers
import torch
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from transformers import DistilBertTokenizer, DistilBertModel
import logging
logging.basicConfig(level=logging.ERROR)

In [10]:
# # Setting up the device for GPU usage

from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
device

'cuda'

In [11]:
def hamming_score(y_true, y_pred, normalize=True, sample_weight=None):
    acc_list = []
    for i in range(y_true.shape[0]):
        set_true = set( np.where(y_true[i])[0] )
        set_pred = set( np.where(y_pred[i])[0] )
        tmp_a = None
        if len(set_true) == 0 and len(set_pred) == 0:
            tmp_a = 1
        else:
            tmp_a = len(set_true.intersection(set_pred))/\
                    float( len(set_true.union(set_pred)) )
        acc_list.append(tmp_a)
    return np.mean(acc_list)

In [ ]:
"""load dataset

name: Bhuvaneshwari/intent_classification
from: https://huggingface.co/datasets/Bhuvaneshwari/intent_classification
"""
from datasets import load_dataset
dataset = load_dataset("Bhuvaneshwari/intent_classification")
dataset

In [12]:
"""load dataset

name: mk9165/ko-voicefishing-classification
from: https://huggingface.co/datasets/mk9165/ko-voicefishing-classification
"""
from datasets import load_dataset
dataset = load_dataset("mk9165/ko-voicefishing-classification")
dataset

Generating train split:   0%|          | 0/1012 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 1012
    })
})

In [15]:
# dataset['validation'][:5]

In [14]:
train_filepath = "dataset/dataset.csv"
dataset['train'].to_csv(train_filepath)

Creating CSV from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

2618249

In [29]:
df = pd.read_csv(train_filepath)
df

,text,label
0,"그래요, 어? 일단 차를 알겠고요. 저희들이 kb 금융 기회를 kb 저축은행 인데요...",1
1,"네 고객님 감사합니다. 네 아, 근데 제가 지난 보니까는 없으시다고 하던데요? 어느...",1
2,"그래요, 어? 일단 차를 알겠고요. 저희들이 kb 금융 기회를 kb 저축은행 인데요...",1
3,"네 고객님 감사합니다. 네 아, 근데 제가 지난 보니까는 없으시다고 하던데요? 어느...",1
4,예예예 아이고 통해 보셨어요? 의원님 아 예 좋아했는데요. 이게 좀 이상해요. 내가...,1
...,...,...
1007,#@이름#야 직방에 보니까 #@기타# 2억1천짜리도 있네~ 1500도있구? 그르킨 ...,0
1008,거긴 맛있어 ?? 맛있으면 디저트도 ㄱㄱ ㄴㄴ밥먹고옴 구럼 뭐먹고있뉘 음메리카노 라...,0
1009,인터넷 뭐 깔러 온대요? 므또새로깐대요? 내일 기사 방문하기로 했다 오후1시 에서 ...,0
1010,자기 웅~~~ 나는 여기 김치찜 남은거랑 컵라면이랑 계란먹어 뭔가 캠핑온거 같아 ㅎ...,0


In [17]:
# labels = set(df["label"].values.tolist())
# labels_dict = dict()
# for i, label in enumerate(labels):
#     labels_dict[label] = str(i)
# labels_dict

{0: '0', 1: '1'}

In [20]:
# df['label'] = df['label'].apply(lambda x: labels_dict[x])


In [30]:
# label_df = pd.get_dummies(df['labels'], dtype=int)
onehot_df = pd.get_dummies(df['label'], prefix='label', dtype=int)
sorted_colums = sorted(onehot_df.columns, key=lambda x: int(x.split('_')[1]))
onehot_df = onehot_df[sorted_colums]
onehot_df

,label_0,label_1
0,0,1
1,0,1
2,0,1
3,0,1
4,0,1
...,...,...
1007,1,0
1008,1,0
1009,1,0
1010,1,0


In [31]:
df['labels'] = onehot_df.values.tolist()
df

,text,label,labels
0,"그래요, 어? 일단 차를 알겠고요. 저희들이 kb 금융 기회를 kb 저축은행 인데요...",1,"[0, 1]"
1,"네 고객님 감사합니다. 네 아, 근데 제가 지난 보니까는 없으시다고 하던데요? 어느...",1,"[0, 1]"
2,"그래요, 어? 일단 차를 알겠고요. 저희들이 kb 금융 기회를 kb 저축은행 인데요...",1,"[0, 1]"
3,"네 고객님 감사합니다. 네 아, 근데 제가 지난 보니까는 없으시다고 하던데요? 어느...",1,"[0, 1]"
4,예예예 아이고 통해 보셨어요? 의원님 아 예 좋아했는데요. 이게 좀 이상해요. 내가...,1,"[0, 1]"
...,...,...,...
1007,#@이름#야 직방에 보니까 #@기타# 2억1천짜리도 있네~ 1500도있구? 그르킨 ...,0,"[1, 0]"
1008,거긴 맛있어 ?? 맛있으면 디저트도 ㄱㄱ ㄴㄴ밥먹고옴 구럼 뭐먹고있뉘 음메리카노 라...,0,"[1, 0]"
1009,인터넷 뭐 깔러 온대요? 므또새로깐대요? 내일 기사 방문하기로 했다 오후1시 에서 ...,0,"[1, 0]"
1010,자기 웅~~~ 나는 여기 김치찜 남은거랑 컵라면이랑 계란먹어 뭔가 캠핑온거 같아 ㅎ...,0,"[1, 0]"


In [32]:
df.drop(['label'], axis=1, inplace=True)
new_df = df
new_df

,text,labels
0,"그래요, 어? 일단 차를 알겠고요. 저희들이 kb 금융 기회를 kb 저축은행 인데요...","[0, 1]"
1,"네 고객님 감사합니다. 네 아, 근데 제가 지난 보니까는 없으시다고 하던데요? 어느...","[0, 1]"
2,"그래요, 어? 일단 차를 알겠고요. 저희들이 kb 금융 기회를 kb 저축은행 인데요...","[0, 1]"
3,"네 고객님 감사합니다. 네 아, 근데 제가 지난 보니까는 없으시다고 하던데요? 어느...","[0, 1]"
4,예예예 아이고 통해 보셨어요? 의원님 아 예 좋아했는데요. 이게 좀 이상해요. 내가...,"[0, 1]"
...,...,...
1007,#@이름#야 직방에 보니까 #@기타# 2억1천짜리도 있네~ 1500도있구? 그르킨 ...,"[1, 0]"
1008,거긴 맛있어 ?? 맛있으면 디저트도 ㄱㄱ ㄴㄴ밥먹고옴 구럼 뭐먹고있뉘 음메리카노 라...,"[1, 0]"
1009,인터넷 뭐 깔러 온대요? 므또새로깐대요? 내일 기사 방문하기로 했다 오후1시 에서 ...,"[1, 0]"
1010,자기 웅~~~ 나는 여기 김치찜 남은거랑 컵라면이랑 계란먹어 뭔가 캠핑온거 같아 ㅎ...,"[1, 0]"


In [ ]:
# data = pd.read_csv('train.csv')
# data

In [43]:
# Sections of config

# Defining some key variables that will be used later on in the training
MAX_LEN = 128
TRAIN_BATCH_SIZE = 4
VALID_BATCH_SIZE = 4
EPOCHS = 1
LEARNING_RATE = 1e-05

In [34]:
tokenizer = DistilBertTokenizer.from_pretrained('distilbert/distilbert-base-multilingual-cased', truncation=True, do_lower_case=True)

In [35]:
class MultiLabelDataset(Dataset):

    def __init__(self, dataframe, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.text = dataframe.text
        self.targets = self.data.labels
        self.max_len = max_len

    def __len__(self):
        return len(self.text)

    def __getitem__(self, index):
        text = str(self.text[index])
        text = " ".join(text.split())

        inputs = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_token_type_ids=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]

        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'targets': torch.tensor(self.targets[index], dtype=torch.float)
        }

In [36]:
# Creating the dataset and dataloader for the neural network
train_size = 0.8
train_data=new_df.sample(frac=train_size,random_state=200)
test_data=new_df.drop(train_data.index).reset_index(drop=True)
train_data = train_data.reset_index(drop=True)

print("FULL Dataset: {}".format(new_df.shape))
print("TRAIN Dataset: {}".format(train_data.shape))
print("TEST Dataset: {}".format(test_data.shape))

training_set = MultiLabelDataset(train_data, tokenizer, MAX_LEN)
testing_set = MultiLabelDataset(test_data, tokenizer, MAX_LEN)
# training_set = MultiLabelDataset(train_data, tokenizer)
# testing_set = MultiLabelDataset(test_data, tokenizer)

FULL Dataset: (1012, 2)
TRAIN Dataset: (810, 2)
TEST Dataset: (202, 2)


In [71]:
print(test_data[:10])
print(test_data[-10:])

                                                text  labels
0  그래요, 어? 일단 차를 알겠고요. 저희들이 kb 금융 기회를 kb 저축은행 인데요...  [0, 1]
1  그래요, 어? 일단 차를 알겠고요. 저희들이 kb 금융 기회를 kb 저축은행 인데요...  [0, 1]
2  여보세요 네. 네 아 네 어저께 통화했던 최고 아닙니다. 어떻게 가족의 결과 안내?...  [0, 1]
3  여보세요? 안녕하세요, 고객님. 김종현 대리 입니다. 아 예 대리님, 예 고객님, ...  [0, 1]
4  에 가정 내 여보세요? 아 내고 있는 선호주였던 하나 기타 김민기 드립니다. 아 네...  [0, 1]
5  네, 혹시 진행이 어떻게 들어가고 있으신가요? 아, 아직 본사 신이 전화를 못 받았...  [0, 1]
6  여보세요 님? 네, 그 저희 쪽 국민은행 쪽에서 의 요청이 들어와서 연락드렸는데요....  [0, 1]
7  여보세요? 네네 아까 전에 통화 도움 드렸던 담당자 이 세리에 요 네네네 저희 고객...  [0, 1]
8  네 여보세요 네 여보세요. 네 아 예 선생님 일단 확인 됐구요. 그 발급 비용 같은...  [0, 1]
9  안녕하세요. 예 여보세요 여보세요 입니다. 고객님 어 지금 전산 내려가지고 연락을 ...  [0, 1]
                                                  text  labels
192  플립은 전용보험써야한대 그래서 6300원.. 엥 그런것도 있음? 응 그렇다네..? ...  [1, 0]
193  ㅋㅋㅋㅋ카페왓는ㄷ0 #@이름#언니랑 #@이름#언니만남 바로뒤에있었어.... ㅌㅋㅋㅋ...  [1, 0]
194  오늘 뭐 묵고싶어?ㅋ 글쎄..겹살에 쏘주를해야하나..회식을해봤어야알지 ㅋㅋ 여봉이 ...  [1, 0]
195  다 배우면 지들 오빠 가게에서 네일샵하는거임? 아니아니 이제는 파트타임 직원으로 하...  [1, 0]
196  어디묘? 지내동 

In [69]:
print(test_data[:10]['text'][0])
print(test_data[:10]['labels'][0])

그래요, 어? 일단 차를 알겠고요. 저희들이 kb 금융 기회를 kb 저축은행 인데요. 지금 5월 달에 저희들이 대환 대출 플러스 생활의 자금 쪽으로 나온 대출상품 있어요. 어 네, 그리고 이은 2월 이 7.8% 대로 진행하는 부분이고 상환기간이 운행 가지고요. 중도상환 가능하고 수수료 발생이 없이 원리금 균등 분할상환 혹은 만기일시상환 쪽으로 이자만 갚을 해도 되는 부분이에요. 네, 그리고 최근 맥심 대환대출 플러스 생활자금 쪽으로 오 천만원까지 인데 오천만원 검토 다 받으실 거예요? 일단 잘 알고 있구요, 지금? 맞으시죠? 예 고객님, 혹시 지금? 그 사용하고 있는 휴대폰은 본인 이름으로 돼 있는 거 맞으세요? 어? 이 통신사는 skt lg 예요. 예 삼성 스마트폰 맞으세요? 혹시 카톡을 사용하고 계세요? 그러면은요 카톡 추가를 해서 저희들 회사 상하고 제 이름 넣어드릴게요. 왜요? 예, 왜요? 예 고객님, 저한테 전화를 주셔서 통화 중에? 제가 전화를 못 받으면 카톡으로 이 연락을 할 수도 있을 있어서 그런 거 아니에요? 아니에요? 네 전화를 주시면?
[0, 1]


In [72]:
print(test_data[-10:].reset_index(drop=True)['text'][0])
print(test_data[-10:].reset_index(drop=True)['labels'][0])

플립은 전용보험써야한대 그래서 6300원.. 엥 그런것도 있음? 응 그렇다네..? 개양아치야.. 방금 가입할라고 들어가니까 극게그거지.. 6300원밖에ㅜ없더라 ㅠ
[1, 0]


In [37]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)

# Creating the Neural Network for Fine Tuning

In [45]:
# Creating the customized model, by adding a drop out and a dense layer on top of distil bert to get the final output for the model.
class DistilBERTClass(torch.nn.Module):
    def __init__(self):
        super(DistilBERTClass, self).__init__()
        # self.l1 = DistilBertModel.from_pretrained("distilbert-base-uncased")
        self.l1 = DistilBertModel.from_pretrained("distilbert/distilbert-base-multilingual-cased")
        self.pre_classifier = torch.nn.Linear(768, 768)
        self.dropout = torch.nn.Dropout(0.1)
        # self.classifier = torch.nn.Linear(768, 12)
        self.classifier = torch.nn.Linear(768, 2)

    def forward(self, input_ids, attention_mask, token_type_ids):
        output_1 = self.l1(input_ids=input_ids, attention_mask=attention_mask)
        hidden_state = output_1[0]
        pooler = hidden_state[:, 0]
        pooler = self.pre_classifier(pooler)
        pooler = torch.nn.Tanh()(pooler)
        pooler = self.dropout(pooler)
        output = self.classifier(pooler)
        return output

In [46]:
model = DistilBERTClass()
model.to(device)

DistilBERTClass(
  (l1): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
            (lin1): Linear(i

In [47]:
def loss_fn(outputs, targets):
    return torch.nn.BCEWithLogitsLoss()(outputs, targets)

In [48]:
optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)

In [49]:
def train(epoch):
    model.train()
    for _,data in tqdm(enumerate(training_loader, 0)):
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.float)

        outputs = model(ids, mask, token_type_ids)

        optimizer.zero_grad()
        loss = loss_fn(outputs, targets)
        if _%5000==0:
            print(f'Epoch: {epoch}, Loss:  {loss.item()}')

        loss.backward()
        optimizer.step()

In [50]:
for epoch in range(EPOCHS):
    train(epoch)

2it [00:00, 18.06it/s]

Epoch: 0, Loss:  0.6766570806503296


203it [00:05, 38.81it/s]


# Validating Model

In [51]:
def validation(testing_loader):
    model.eval()
    fin_targets=[]
    fin_outputs=[]
    with torch.no_grad():
        for _, data in tqdm(enumerate(testing_loader, 0)):
            ids = data['ids'].to(device, dtype = torch.long)
            mask = data['mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
            targets = data['targets'].to(device, dtype = torch.float)
            outputs = model(ids, mask, token_type_ids)
            fin_targets.extend(targets.cpu().detach().numpy().tolist())
            fin_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())
    return fin_outputs, fin_targets

In [53]:
output, targets = validation(testing_loader)
print(f"outputs:\n{output[:10]}")
print(f"targets:\n{targets[:10]}")

final_outputs = np.array(output) >=0.5
print(f"final outputs:\n{final_outputs[:10]}")


51it [00:01, 48.41it/s]

outputs:
[[0.015593708492815495, 0.9839683771133423], [0.012718467973172665, 0.9867292642593384], [0.987812340259552, 0.01133679784834385], [0.015973716974258423, 0.9837032556533813], [0.9867156147956848, 0.01211388036608696], [0.9875983595848083, 0.011204879730939865], [0.9871152639389038, 0.013136222027242184], [0.9881227612495422, 0.011964518576860428], [0.01599281094968319, 0.9834138751029968], [0.013309494592249393, 0.986107587814331]]
targets:
[[0.0, 1.0], [0.0, 1.0], [1.0, 0.0], [0.0, 1.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [0.0, 1.0], [0.0, 1.0]]
final outputs:
[[False  True]
 [False  True]
 [ True False]
 [False  True]
 [ True False]
 [ True False]
 [ True False]
 [ True False]
 [False  True]
 [False  True]]


In [54]:
import numpy as np
list(map(lambda x: np.where(x == True)[0].item(), final_outputs[:10]))

[1, 1, 0, 1, 0, 0, 0, 0, 1, 1]

In [55]:
val_hamming_loss = metrics.hamming_loss(targets, final_outputs)
val_hamming_score = hamming_score(np.array(targets), np.array(final_outputs))

print(f"Hamming Score = {val_hamming_score}")
print(f"Hamming Loss = {val_hamming_loss}")

Hamming Score = 0.995049504950495
Hamming Loss = 0.0049504950495049506


# Saving the files for inference

In [56]:
from datetime import datetime
today = datetime.now().strftime("%Y%m%d")
output_model_file = './output/pytorch_distilbert_{}.bin'.format(today)
output_vocab_file = './output/vocab_distilbert_{}.bin'.format(today)

torch.save(model, output_model_file)
tokenizer.save_vocabulary(output_vocab_file)

print('Saved')

Saved
